# Crossmatch with ZTF and PanSTARRS

Crossmatch the ComCam data with ZTF and PanSTARRS and store the results as HATS catalogs on disk.

In [ ]:
import os
import lsdb
import tempfile

from upath import UPath
from dask.distributed import Client
from nested_pandas import NestedDtype

In [ ]:
#DRP_VERSION = os.environ["DRP_VERSION"]
DRP_VERSION = "w_2025_06"
print(f"DRP_VERSION: {DRP_VERSION}")
base_output_dir = UPath(f"/sdf/data/rubin/shared/lsdb_commissioning")
hats_dir = base_output_dir / "hats" / DRP_VERSION

In [ ]:
tmp_path = tempfile.TemporaryDirectory()
tmp_dir = tmp_path.name
client = Client(n_workers=16, threads_per_worker=1, local_directory=tmp_dir)

Let's load the nested Rubin catalogs:

In [ ]:
diaObject_lc = lsdb.read_hats(hats_dir / "diaObject_lc").map_partitions(
    lambda df: df.assign(
        **{
            lc_column: df[lc_column].astype(
                NestedDtype.from_pandas_arrow_dtype(df.dtypes[lc_column])
            )
            for lc_column in ["diaSource", "diaForcedSource"]
        }
    )
)
diaObject_lc

In [ ]:
object_lc = lsdb.read_hats(hats_dir / "object_lc").map_partitions(
    lambda df: df.assign(
        **{
            "forcedSource": df["forcedSource"].astype(
                NestedDtype.from_pandas_arrow_dtype(df.dtypes["forcedSource"])
            )
        }
    )
)
object_lc

Let's load ZTF DR22 with lightcurves and PanSTARRS:

In [ ]:
ztf_dr22_lc = lsdb.read_hats(
    "https://data.lsdb.io/hats/ztf_dr22/ztf_lc",
    margin_cache="https://data.lsdb.io/hats/ztf_dr22/ztf_lc_10arcs",
)
ztf_dr22_lc

In [ ]:
panstarrs = lsdb.read_hats(
    UPath("s3://stpubdata/panstarrs/ps1/public/hats/otmo", anon=True),
    margin_cache=UPath(
        "s3://stpubdata/panstarrs/ps1/public/hats/otmo_10arcs", anon=True
    ),
)
panstarrs

### Crossmatch with PanSTARRS

In [ ]:
ztf_xmatch_radius_arcsec = 0.2

In [ ]:
for catalog in [diaObject_lc, object_lc]:
    catalog_name = f"{catalog.name}_x_ztf_dr22"
    lsst_lc_x_ztf_dr22 = catalog.crossmatch(
        ztf_dr22_lc, radius_arcsec=ztf_xmatch_radius_arcsec, suffixes=("", "_ztf")
    )
    lsst_lc_x_ztf_dr22._ddf = lsst_lc_x_ztf_dr22._ddf.rename(columns={"_dist_arcsec": "lsst_ztf_sep"})
    lsst_lc_x_ztf_dr22.to_hats(hats_dir / catalog_name, catalog_name=catalog_name)
    print(f"Saved {catalog_name}")

### Crossmatch with PS1

In [ ]:
ps1_xmatch_radius_arcsec = 0.1

In [ ]:
for catalog in [diaObject_lc, object_lc]:
    catalog_name = f"{catalog.name}_x_ps1"
    lsst_lc_x_ps1_dr22 = catalog.crossmatch(
        panstarrs, radius_arcsec=ps1_xmatch_radius_arcsec, suffixes=("", "_ps1")
    )
    lsst_lc_x_ps1_dr22._ddf = lsst_lc_x_ps1_dr22._ddf.rename(columns={"_dist_arcsec": "lsst_ps1_sep"})
    lsst_lc_x_ps1_dr22.to_hats(hats_dir / catalog_name, catalog_name=catalog_name)
    print(f"Saved {catalog_name}")

In [ ]:
client.close()
tmp_path.cleanup()